# HyperPerimeter Tuning for the above model...
## pip install keras-tuner

In [1]:
import sqlite3 as sql
from keras.src.models import Sequential 
from keras.src.layers import Dense, Dropout, Input
from keras_tuner import RandomSearch 
from sklearn.preprocessing import LabelEncoder
from keras.src.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.src.utils import to_categorical
import pandas as pd 

In [4]:
## Spliting data into train and test data

## Read the raw data from sql table.

conn = sql.connect('../DataBase/TrainingData.db')
query="SELECT * FROM HealthData"
df = pd.read_sql_query(query, conn)

## Selecting all columns which have type = object.
cat_col=[col for col in df.columns if df[col].dtype == 'object']

encoder=LabelEncoder()
for col in cat_col:
    
    df[col]=encoder.fit_transform(df[col])

X=df.iloc[:,:-1]
y=df['NObeyesdad']

x_train_full,  x_test, y_train_full , y_test =train_test_split(X,y, test_size=0.2, random_state=42)

x_train_hp, x_valid, y_train_hp, y_valid = train_test_split(x_train_full, y_train_full, random_state=42, test_size=0.2)


In [5]:
## Scaling the data by using StandardScaler

hp_scaler = StandardScaler()

x_train_scaled_hp = hp_scaler.fit_transform(x_train_hp)
x_test_scaled_hp = hp_scaler.transform(x_valid)
x_test_evlve= hp_scaler.transform(x_test)

In [6]:
x_test_scaled_hp.shape

(338, 16)

In [7]:
## Converting  the target data into One-Hot vector

y_train_hp_cat = to_categorical(y_train_hp)
y_valid_cat = to_categorical(y_valid, 7)
y_test_hp = to_categorical(y_test,7)
y_valid_cat.shape

(338, 7)

In [11]:
## Define the model-building function 

def build_model(hp):
    
    model = Sequential()
    
    # Input layer with tunable number of neurone
    model.add(Input(shape=(x_train_scaled_hp.shape[1],)))
    model.add(Dense(units=hp.Int('units_input', min_value=32, max_value=256, step=32),
                    activation='relu'))
    
    # first hidden layer with tunable units and dropout
    model.add(Dense(units=hp.Int('units_hidden1', min_value=64, max_value=256, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_hidden1', min_value=0.2, max_value=0.5, step=0.1)))
    
    # second hidden layer
    model.add(Dense(units=hp.Int('unit_hidden2', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('unit_dropout2', min_value=0.2, max_value=0.5, step=0.1)))
    
    # third hidden layer 
    model.add(Dense(units=hp.Int('unit_hidden3', min_value=16, max_value=64, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('unit_dropout3', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Output layer
    model.add(Dense(y_train_hp_cat.shape[1], activation='softmax'))
    
    # Compie the model with a tunable learning rate
    model.compile(optimizer=Adam( learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [20]:
## Initialize the tuner 

tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,
                     executions_per_trial=3,
                     directory='hyperparameter_tuning',
                     project_name='Obesity Classification')

In [21]:
## Start the search using your training data and validation data.

tuner.search(x_train_scaled_hp, y_train_hp_cat, epochs=100, validation_data=(x_test_scaled_hp, y_valid_cat))

Trial 5 Complete [00h 03m 02s]
val_accuracy: 0.9309664567311605

Best val_accuracy So Far: 0.9428008000055949
Total elapsed time: 00h 07m 42s


In [22]:
## Selecting the best model.

best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best hyperparameters:", best_hyperparameters.values)


Best hyperparameters: {'units_input': 64, 'units_hidden1': 128, 'dropout_hidden1': 0.4, 'unit_hidden2': 32, 'unit_dropout2': 0.30000000000000004, 'unit_hidden3': 48, 'unit_dropout3': 0.30000000000000004, 'learning_rate': 0.01}


d:\Anaconda3\envs\Obesity_venv\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [25]:
## evaluate on test data
test_loss, test_accuracy = best_model.evaluate(x_test_evlve, y_test_hp)
print(f"Test accuracy of the best model: {test_accuracy:.2f}")


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9369 - loss: 0.3870
Test accuracy of the best model: 0.93


<table border="1">
  <thead>
    <tr>
      <th>Parameter</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>units_input</td>
      <td>64</td>
    </tr>
    <tr>
      <td>units_hidden1</td>
      <td>128</td>
    </tr>
    <tr>
      <td>dropout_hidden1</td>
      <td>0.4</td>
    </tr>
    <tr>
      <td>unit_hidden2</td>
      <td>32</td>
    </tr>
    <tr>
      <td>unit_dropout2</td>
      <td>0.3</td>
    </tr>
    <tr>
      <td>unit_hidden3</td>
      <td>48</td>
    </tr>
    <tr>
      <td>unit_dropout3</td>
      <td>0.3</td>
    </tr>
    <tr>
      <td>learning_rate</td>
      <td>0.01</td>
    </tr>
  </tbody>
</table>
